In [90]:
import torch.nn    as nn
import torch.optim as optim
import GM_library  as GML
import numpy       as np
import torch
import json

from os                   import path, listdir
from torch.utils.data     import random_split
from torch_geometric.data import Data

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Target to generate new crystals
target = 'GM_BiSI'

# In case database is created from scratch (otherwise, it is not being used)
DB_path = '../MP/Loaded_PT'

input_folder    = 'models'
target_folder   = f'{input_folder}/{target}'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

In [3]:
# Define target value to look for
#seeked_target = ##

# Machine-learning parameters
n_epochs      = 300
batch_size    = 32
learning_rate = 0.0001

# Ratios for dividing training data
test_ratio = 0.0

# Number of diffusing and denoising steps
n_t_steps = 10

# Decay of parameter alpha
noise_contribution = 0.15
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = 0.2
dropout_edge = 0.2

# Create and save as a dictionary
model_parameters = {
    'n_epochs': n_epochs,
    'batch_size': batch_size,
    'learning_rate': learning_rate,
    'test_ratio': test_ratio,
    'n_t_steps': n_t_steps,
    'noise_contribution': noise_contribution,
    'dropout_node': dropout_node,
    'dropout_edge': dropout_edge
}

# Write the dictionary to the file in JSON format
with open(f'{target_folder}/model_parameters.json', 'w') as json_file:
    json.dump(model_parameters, json_file)

Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556], y=[1, 1])

In [79]:
dataset[0].edge_attr

tensor([0.8986, 0.6007, 0.0000,  ..., 0.3057, 0.0000, 0.0000])

In [78]:
GML.POSCAR_graph_encoding(dataset[0], L, file_name=f'CONTCAR', POSCAR_directory='./')

Invalid graph, atoms overlapping. Applying brute force :)
14 41 58 6.8787885 14.762562 12.048041 8.72941957421743 11.905060467711378


<_io.TextIOWrapper name='.//CONTCAR' mode='w' encoding='UTF-8'>

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [52]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

from pymatgen.io.vasp.inputs  import Poscar
from pymatgen.io.vasp.outputs import Vasprun

import sys
sys.path.append('../../UCL/m3gnet')
import ML_library as MLL

dataset = []
labels  = []

### Loaded database ###

data_path = 'Loaded_BiSI/gamma'

# Iterate over materials and relaxations in the dataset
while len(dataset) < 200:
    for material in listdir(data_path):
        # Define path to material
        path_to_material = f'{data_path}/{material}'

        # Check if it is a folder
        if not path.isdir(path_to_material):
            continue

        print()
        print(material)

        # Get relaxations steps (rel1, rel2...)
        relaxation_steps = listdir(path_to_material)

        # Determine all defect states across every folder
        defect_states = []
        for relaxation_step in relaxation_steps:
            path_to_relaxation_step = f'{path_to_material}/{relaxation_step}'
            if path.isdir(path_to_relaxation_step):
                for defect_state in listdir(path_to_relaxation_step):
                    if path.isdir(f'{path_to_material}/{relaxation_step}/{defect_state}'):
                        defect_states.append(defect_state)

        # Determine unique defect states across every folder
        unique_defect_states = np.unique(defect_states)

        # Run over all defect states
        for defect_state in unique_defect_states:
            print(f'\t{defect_state}')

            # Run over all relaxation steps
            for relaxation_step in relaxation_steps:
                # Define path to relaxation loading every relaxation step of a same defect state in the same data column
                path_to_deformation = f'{path_to_material}/{relaxation_step}/{defect_state}'

                # Avoiding non-directories (such as .DS_Store)
                if not path.isdir(path_to_deformation):
                    continue

                # Define name for the defect state folder
                temp_relaxation = f'{material}_{defect_state}'

                # Check if it is a valid relaxation (with a vasprun.xml file)
                # If not, it might be that there are different deformation folders of the defect state
                if MLL.is_relaxation_folder_valid(path_to_deformation):
                    path_to_relaxations = [path_to_deformation]
                else:
                    # Try to extact deformation folders
                    deformation_folders = listdir(path_to_deformation)

                    # Run over deformations
                    path_to_relaxations = []
                    for deformation_folder in deformation_folders:
                        path_to_relaxation = f'{path_to_deformation}/{deformation_folder}'
                        if MLL.is_relaxation_folder_valid(path_to_relaxation):
                            path_to_relaxations.append(path_to_relaxation)

                # Gather relaxations from different deformations as different ionic steps
                for path_to_relaxation in path_to_relaxations:
                    # Remove invalid characters from the vasprun.xml file
                    MLL.clean_vasprun(path_to_relaxation)  # Uncomment is it happens to you as well!!

                    if not path.exists(f'{path_to_relaxation}/vasprun.xml'):
                        print(f'Check {path_to_relaxation}')

                    # Load data from relaxation
                    try:
                        # Try to load those unfinished relaxations as well
                        vasprun = Vasprun(f'{path_to_relaxation}/vasprun.xml', exception_on_bad_xml=False)
                    except:
                        print('Error: vasprun not correctly loaded.')
                        continue

                    # Run over ionic steps
                    for ionic_step_idx in range(len(vasprun.ionic_steps)):
                        temp_ionic_step = f'{temp_relaxation}_{ionic_step_idx}'
                        # Extract data from each ionic step
                        temp_structure = vasprun.ionic_steps[ionic_step_idx]['structure']
                        temp_energy    = vasprun.ionic_steps[ionic_step_idx]['e_fr_energy']

                        # Save temp_structure to POSCAR
                        temp_Poscar = Poscar(temp_structure)
                        temp_Poscar.write_file('POSCAR')

                        # Extract parameters from POSCAR
                        cell, composition, concentration, positions = MPL.information_from_VASPfile('.',
                                                                                                    'POSCAR')

                        # Generate POSCAR covering the box
                        try:
                            nodes, edges, attributes, _, _, _ = GML.graph_POSCAR_encoding(cell,
                                                                                          composition,
                                                                                          concentration,
                                                                                          positions,
                                                                                          L)
                        except:
                            print(f'Error: ionic step {ionic_step_idx} not loaded')
                            continue

                        # Construct temporal graph structure
                        graph = Data(x=nodes,
                                     edge_index=edges,
                                     edge_attr=attributes,
                                     y=torch.tensor([[temp_energy]], dtype=torch.float)
                                    )

                        # Append to dataset and labels
                        dataset.append(graph)
                        labels.append(f'')


BiSBr

BiSeBr

BiSeI

BiSI
	as_1_Bi_on_S_-1
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
	as_1_Bi_on_S_-2
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
	as_1_Bi_on_S_0
	as_1_Bi_on_S_1
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
	as_1_Bi_on_S_2
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
	as_1_Bi_on_S_3
Error: vasprun not correctly loaded.
	as_1_Bi_on_S_4
	as_1_Bi_on_S_5
	as_1_I_on_Bi_-1
Error: vasprun not correctly loaded.
Error: vasprun not correctly loaded.
Error: vasp

In [54]:
# Standardize dataset
dataset, parameters = GML.standardize_dataset(dataset)

# Save standardized dataset
torch.save(dataset,    dataset_name_std)
torch.save(parameters, parameters_name_std)
torch.save(labels,     labels_name)

labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

if path.exists(dataset_name_std) and path.exists(parameters_name_std) and path.exists(labels_name):
    # Load the standardized dataset, with corresponding labels and parameters
    dataset    = torch.load(dataset_name_std)
    labels     = torch.load(labels_name)
    parameters = torch.load(parameters_name_std)

    # Assigning parameters accordingly
    target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale = parameters
    
    # Defining target factor
    target_factor = target_std / scale

elif path.exists(dataset_name) and path.exists(labels_name):
    # Load the raw dataset, with corresponding labels, and standardize it
    dataset = torch.load(dataset_name)
    labels  = torch.load(labels_name)
    
    # Standardize dataset
    dataset, parameters = GML.standardize_dataset(dataset)
    
    # Save standardized dataset
    torch.save(dataset,    dataset_name_std)
    torch.save(parameters, parameters_name_std)

else:
    # Generate the raw dataset from scratch, and standardize it
    
    # Read all mateials within the database
    materials = listdir(DB_path)[:1000]
    
    dataset = []
    labels  = []
    for material in materials:
        try:
            # Try to read the polyforms
            polymorfs = listdir(f'{DB_path}/{material}')
        except:
            continue
        
        print(material)
        for polymorf in polymorfs:
            # Path to folder containing the POSCAR
            path_to_POSCAR = f'{DB_path}/{material}/{polymorf}'
            
            # Check that the folder is valid
            if path.exists(f'{path_to_POSCAR}/POSCAR'):
                print(f'\t{polymorf}')

                # Extract parameters from POSCAR
                cell, composition, concentration, positions = MPL.information_from_VASPfile(path_to_POSCAR,
                                                                                            'POSCAR')

                # Generate POSCAR covering the box
                try:
                    nodes, edges, attributes, _, _, _ = GML.graph_POSCAR_encoding(cell,
                                                                                  composition,
                                                                                  concentration,
                                                                                  positions,
                                                                                  L)
                except:
                    print(f'Error: {material} {polymorf} not loaded')
                    continue

                # Load ground state energy per atom
                gs_energy = float(np.loadtxt(f'{path_to_POSCAR}/EPA'))

                # Construct temporal graph structure
                graph = Data(x=nodes,
                             edge_index=edges,
                             edge_attr=attributes,
                             y=torch.tensor([[gs_energy]], dtype=torch.float)
                            )

                # Append to dataset and labels
                dataset.append(graph)
                labels.append(f'{material}-{polymorf}')
    
    # Standardize dataset
    dataset, parameters = GML.standardize_dataset(dataset)
    
    # Save standardized dataset
    torch.save(dataset,    dataset_name_std)
    torch.save(parameters, parameters_name_std)
    torch.save(labels,     labels_name)

# Definition of train-test datasets

In [55]:
# torch.manual_seed(12345)

# Define the sizes of the train and test sets
test_size  = int(test_ratio * len(dataset))
train_size = len(dataset) - test_size

# Use random_split() to generate train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of testing  graphs: {len(test_dataset)}')

Number of training graphs: 73670
Number of testing  graphs: 0


# Training of the model

In [66]:
import importlib
importlib.reload(GML)

<module 'GM_library' from '/Users/cibran/Work/UPC/GenerativeModels/GM_library.py'>

In [67]:
# Determine number of features in dataset, considering the t_step information
n_features = dataset[0].num_node_features + 1

# Instantiate the models for nodes and edges
node_model = GML.nGCNN(n_features, dropout_node).to(device)
edge_model = GML.eGCNN(n_features, dropout_edge).to(device)

# Load previous model if available
try:
    node_model.load_state_dict(torch.load(node_model_name))
    edge_model.load_state_dict(torch.load(edge_model_name))
except FileNotFoundError:
    pass

# Evaluate model state
node_model.eval()
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(5, 256)
  (conv2): GraphConv(256, 5)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=6, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)


In [68]:
# Initialize the optimizers
node_optimizer = torch.optim.Adam(node_model.parameters(), lr=learning_rate)
edge_optimizer = torch.optim.Adam(edge_model.parameters(), lr=learning_rate)

# Training loop
train_losses = []
for epoch in range(n_epochs):
    # Initialize train loss variable
    train_loss = 0
    graph_counter = 0
    for graph in train_dataset:
        # Count new graph
        graph_counter += 1
        
        #print()
        #print(f'Graph: {graph_counter}')
        # Clone existing graph
        graph_0 = graph.clone()
        
        # Initialize the gradient of the optimizers
        node_optimizer.zero_grad()
        edge_optimizer.zero_grad()
        
        # Start denoising-diffusing process
        for t_step in np.arange(1, n_t_steps+1):
            # Diffuse the graph with some noise
            #print()
            #print(f'Step: {t_step}')
            #print('Diffusing...')
            
            graph_t, epsilon_t = GML.diffusion_step(graph_0, t_step, n_t_steps, alpha_decay)
            
            # Update diffused graph as next one
            graph_0 = graph_t.clone()

            # Denoise the diffused graph
            #print(f'Denoising...')
            
            # Add t_step information to graph_t
            graph_t = GML.add_t_information(graph_t, t_step)

            # Perform a single forward pass for predicting node features
            out_x = node_model(graph_t.x,
                               graph_t.edge_index,
                               graph_t.edge_attr)
            
            # Remove t_step information
            out_x = out_x[:, :-1]

            # Define x_i and x_j as features of every corresponding pair of nodes (same order than attributes)
            x_i = graph_t.x[graph_t.edge_index[0]]
            x_j = graph_t.x[graph_t.edge_index[1]]

            # Perform a single forward pass for predicting edge attributes
            # Introduce previous edge attributes as features as well
            out_attr = edge_model(x_i, x_j, graph_t.edge_attr)

            # Construct noise graph
            pred_epsilon_t = Data(x=out_x,
                                  edge_index=graph_t.edge_index,
                                  edge_attr=out_attr.ravel())
            
            # Backpropagation and optimization step
            #print('Backpropagating...')

            # Calculate the loss for node features and edge attributes
            node_loss, edge_loss = GML.get_graph_losses(epsilon_t, pred_epsilon_t)
            
            # Backpropagate and optimize node loss
            node_loss.backward(retain_graph=True)
            node_optimizer.step()

            # Backpropagate and optimize edge loss
            edge_loss.backward(retain_graph=True)
            edge_optimizer.step()

            # Predict target for current graph
            #predicted_target = ###

            # Compute target loss
            #target_loss = GML.get_target_loss(predicted_target, seeked_target)
            target_loss = 0

            # Accumulate the total training loss
            loss = node_loss + edge_loss + target_loss
            train_loss += loss.item()
    
    # Compute the average train loss
    train_loss = train_loss / (len(train_dataset) * n_t_steps)
    train_losses.append(train_loss)
    
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}')
    
    if (epoch % 20) == 0:
        torch.save(node_model.state_dict(), node_model_name)
        torch.save(edge_model.state_dict(), edge_model_name)

torch.save(node_model.state_dict(), node_model_name)
torch.save(edge_model.state_dict(), edge_model_name)

Epoch: 1, Train Loss: 179278.5708
Epoch: 2, Train Loss: 316.5640
Epoch: 3, Train Loss: 63.3639
Epoch: 4, Train Loss: 7.9131
Epoch: 5, Train Loss: 2.7593
Epoch: 6, Train Loss: 2.1329
Epoch: 7, Train Loss: 2.1127
Epoch: 8, Train Loss: 2.0214
Epoch: 9, Train Loss: 2.0088
Epoch: 10, Train Loss: 2.0163


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.log(train_losses))
plt.xlabel('Epoch')
plt.ylabel('Loss function')